In [ ]:
paths = {
    'ro_sgp': '/home/yishu/open_anything_diffusion/logs/sim_demo_trajectory_pn++/2024-06-23/01-25-15/logs/instance_result.json',
    'ro_raw': '/home/yishu/open_anything_diffusion/logs/sim_demo_trajectory_pn++/2024-06-23/01-25-53/logs/instance_result.json',
    'history': '/home/yishu/open_anything_diffusion/logs/sim_demo_trajectory_diffuser_hispndit/2024-06-23/01-27-32/logs/instance_result.json'
}

In [ ]:
import json

results = {}

for name in paths.keys():
    with open(paths[name], 'r') as f:
        results[name] = json.load(f)

In [ ]:
for instance_id in results['history'].keys():
    if instance_id not in results['ro_sgp'].keys():
        continue

    # if results['history'][instance_id] >= 0.9 and results['ro_sgp'][instance_id] < 0.9:
    if results['history'][instance_id] >= 0.9 and results['ro_sgp'][instance_id] < 0.3:
        print(instance_id)

### Flow visualization

In [ ]:
id = '100444_link_0'

In [ ]:
import pickle as pkl
orig_path = f'/home/yishu/open_anything_diffusion/logs/sim_demo_trajectory_pn++/2024-06-23/01-25-15/logs/flow_vis/{id}.pkl'
hd_path = f'/home/yishu/open_anything_diffusion/logs/sim_demo_trajectory_diffuser_hispndit/2024-06-23/01-27-32/logs/flow_vis/{id}.pkl'

with open(orig_path, 'rb') as f:
    orig_flow = pkl.load(f)

with open(hd_path, 'rb') as f:
    hd_flow = pkl.load(f)

In [ ]:
orig_flow.show()

In [ ]:
hd_flow.show()

## Website grasp visualization

In [ ]:
import pickle
# with open('data_flowbotHD_8877_link2.pkl', 'rb') as f:
# with open('data_flowbot3D_9388_link0.pkl', 'rb') as f:
with open('data_flowbotHD_11712_link0.pkl', 'rb') as f:
# with open('data_flowbotHD_49062_link2.pkl', 'rb') as f:
    data = pickle.load(f)
    # p({
    #     'flow_visual': trial_figs['link_2'],
    #     'grasp_visual': animation,
    flow_visual = data['flow_visual']
    visual_all_points = data['visual_points']
    visual_link_ixs = data['visual_link_ixs']
    visual_grasp_points_idx = data['visual_grasp_points_idx']
    visual_grasp_points = data['visual_grasp_points']
    visual_flows = data['visual_flow']

In [ ]:
# flow_visual.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os

# path = './grasp_visuals/hd_8877link2/'
# path = './grasp_visuals/3d_9388link0/'
path = './grasp_visuals/hd_11712link0/'
# path = './grasp_visuals/hd_49062link2/'
if not os.path.exists(path):
    os.makedirs(path)

for step in range(0, len(visual_all_points)):
# for step in range(16, 17):
    points, link_ixs, grasp_point_idx, grasp_point, flow = visual_all_points[step], visual_link_ixs[step], visual_grasp_points_idx[step], visual_grasp_points[step], visual_flows[step]
    num_points = 1200

    # Color part of the points red and others blue
    colors = np.array(['blue'] * (num_points))
    colors[link_ixs] = 'red'

    # Grasp arrow
    arrow_origin = np.array(grasp_point)
    arrow_direction = np.array(flow / np.linalg.norm(flow)) * 0.5
    arrow_end = arrow_origin + arrow_direction
    cone_size = arrow_direction * 0.8

    # Create a scatter plot for the point cloud
    scatter = go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color=colors,
            opacity=0.2
        )
    )


    arrow = go.Scatter3d(
        x=[arrow_origin[0], arrow_end[0]],
        y=[arrow_origin[1], arrow_end[1]],
        z=[arrow_origin[2], arrow_end[2]],
        mode='lines+markers',
        line=dict(color='green', width=10),
        marker=dict(size=3, color='black')
    )

    # cone = go.Cone(x=[arrow_end[0]], y=[arrow_end[1]], z=[arrow_end[2]], u=[(arrow_end[0]-arrow_origin[0]) * 0.7], v=[(arrow_end[1]-arrow_origin[1]) * 0.7], w=[(arrow_end[2]-arrow_origin[2]) * 0.7], colorscale='Greens', showscale=False)
    cone = go.Cone(x=[arrow_end[0]], y=[arrow_end[1]], z=[arrow_end[2]], u=[cone_size[0]], v=[cone_size[1]], w=[cone_size[2]], colorscale='Greens', showscale=False)

    # Combine the scatter plot and the arrow
    fig = go.Figure(data=[scatter, arrow, cone])
    # fig = go.Figure(data=[scatter])

    camera = dict(
        up=dict(x=0, y=0, z=0.5),

        # # 8877
        # center=dict(x=0, y=0, z=0),
        # eye=dict(x=-0.975, y=-0.45, z=0.875)

        # # 9388
        # center=dict(x=0, y=0, z=-0.1),
        # eye=dict(x=-1.32, y=-0.48, z=0.72)

        # # 49062
        # center=dict(x=0, y=-0.05, z=0.1),
        # eye=dict(x=-0.95, y=0.72, z=0.5184)

        # 11712
        center=dict(x=0, y=-0.1, z=0.1),
        eye=dict(x=-0.99, y=0.72, z=0.54)
    )

    fig.update_layout(
        scene=dict(
            # xaxis=dict(visible=False, range=[-1, 1]),
            # yaxis=dict(visible=False, range=[-1, 1]),
            # zaxis=dict(visible=False, range=[0, 2]),


            xaxis=dict(visible=False, range=[-1.5, 1.5]),
            yaxis=dict(visible=False, range=[-1.5, 1.5]),
            zaxis=dict(visible=False, range=[-1.0, 2.0]),
            
            # xaxis=dict(visible=True, range=[-1, 1]),
            # yaxis=dict(visible=True, range=[-1, 1]),
            # zaxis=dict(visible=True, range=[0, 2]),
            xaxis_showspikes=False,
            yaxis_showspikes=False,
            zaxis_showspikes=False,
            bgcolor='rgba(255,255,255,1)',
            aspectmode = 'cube'
        ),
        width=500,
        height=500,
        showlegend=False,
        coloraxis_showscale=False,
        margin=dict(l=0, r=0, b=0, t=0),
        paper_bgcolor='rgba(0,0,0,0)',
        scene_camera=camera,
    )

    # # Show plot
    # fig.show()
    pio.write_image(fig, f'{path}/{step}.jpeg', format='jpeg')
    # break